<a href="https://colab.research.google.com/github/Soroushav/llm_basics/blob/main/python_to_cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)
MODEL_GPT = "gpt-5-nano"

In [ ]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M4 Macbook air. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [ ]:
def get_prompt_for(python):
  user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
  user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
  user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
  user_prompt += python
  return user_prompt


In [ ]:
def message_for(python):
  return [{
      "role": "system",
      "content": system_message
  }, {
      "role": "user",
      "content": get_prompt_for(python)
  }]

In [ ]:
def write_output(cpp):
  with open("output.cpp", "w") as f:
    f.write(cpp)

In [ ]:
def optimize_for(python):
  stream = openai.chat.completions.create(model=MODEL_GPT, messages=message_for(python), stream=True)
  result = ""
  for chunk in stream:
    content = chunk.choices[0].delta.content or ""
    result += content
    print(content, end='', flush=True)
  write_output(result.replace("```", "").replace("```cpp","").replace("cpp", ""))

In [ ]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
exec(pi)

In [ ]:
optimize_for(pi)

In [ ]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value

def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:
exec(python_hard)

In [ ]:
optimize_for(python_hard)

In [ ]:
def stream_gpt(python):
  stream = openai.chat.completions.create(model=MODEL_GPT, messages=message_for(python), stream=True)
  result = ""
  for chunk in stream:
    content = chunk.choices[0].delta.content or ""
    result += content
    yield result

In [ ]:
with gr.Blocks() as ui:
  gr.Markdown("Your code generator from python to c++")
  with gr.Row():
    python = gr.Textbox(label="Python input:", value=pi, lines=10)
    cpp = gr.Textbox(label="CPP output:", lines=10)
  with gr.Row():
    convert = gr.Button("Convert")
  convert.click(fn=stream_gpt, inputs=[python], outputs=[cpp])
ui.launch()

In [ ]:
OLLAMA_BASE_URL = "https://charmlessly-unbenumbed-matthew.ngrok-free.dev"
ollama = OpenAI(base_url=f"{OLLAMA_BASE_URL}/v1", api_key="ollama")

In [ ]:
def stream_ollama(python):
  stream = ollama.chat.completions.create(model="llama3.2:latest", messages=message_for(python), stream=True)
  result = ""
  for chunk in stream:
    content = chunk.choices[0].delta.content or ""
    result += content
    yield result


In [ ]:
def generate(python, model):
  if model == "GPT":
    result = stream_gpt(python)
  elif model == "OLLAMA":
    result = stream_ollama(python)
  for streamed_so_far in result:
    yield streamed_so_far

In [ ]:
with gr.Blocks() as ui:
  gr.Markdown("Your code generator from python to c++")
  with gr.Row():
    python = gr.Textbox(label="Python input:", value=pi, lines=10)
    cpp = gr.Textbox(label="CPP output:", lines=10)
  with gr.Row():
    model = gr.Dropdown(label="Model", choices=["GPT", "OLLAMA"], value="GPT")
    convert = gr.Button("Convert")
  convert.click(fn=generate, inputs=[python, model], outputs=[cpp])
ui.launch()